In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, Subset
import matplotlib.pyplot as plt
from PIL import Image
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
import copy
import time

In [ ]:
tmfs = transforms.Compose([
                    #transforms.CenterCrop((1000,1000)),
                    #transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0),
                    #transforms.FiveCrop(size),
                    #transforms.Grayscale(num_output_channels=1),
                    #transforms.Pad(padding, fill=0, padding_mode='constant'),
                    #transforms.RandomAffine(degrees, translate=None, scale=None, shear=None, resample=False, fillcolor=0),
                    #transforms.RandomApply(transforms, p=0.5),
                    #transforms.RandomChoice(transforms),
                    #transforms.RandomCrop(size, padding=None, pad_if_needed=False, fill=0, padding_mode='constant'),
                    #transforms.RandomGrayscale(p=0.1),
                    #transforms.RandomHorizontalFlip(p=0.5),
                    #transforms.RandomOrder(transforms),
                    #transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3, fill=0),
                    #transforms.RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
                    #transforms.RandomRotation(45, resample=False, expand=False, center=None, fill=None),
                    #transforms.RandomVerticalFlip(p=0.5),
                    transforms.Resize((224,224), interpolation=2),
                    #transforms.TenCrop(size, vertical_flip=False),
                    
                    #########################################################
                    #transforms.ToPILImage(mode=None),
                    transforms.ToTensor(),
    
                    #########################################################
                    #transforms.LinearTransformation(transformation_matrix, mean_vector),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False),
                    #transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
                    

                    
                    #########################################################
                    #transforms.Lambda(lambd)
                   
                   ])

In [ ]:
df = pd.read_csv('data/ISIC_2019_Training_GroundTruth.csv')
mapping = {e:i for i, e in enumerate(df.iloc[:,1:].idxmax(1).unique())}
df['label_'] = df.iloc[:,1:].idxmax(1)
df['label']  = df['label_'].map(mapping)
df['fold'] = -1

fold_n = 10
skf = KFold(n_splits=fold_n)
for fold, (train_idx, test_idx) in enumerate(skf.split(df['label']), start=1):
    df.loc[test_idx, 'fold'] =  fold
    
debug = False
if debug:
    df.groupby(['label','fold']).head(2).to_csv('data/ISIC_2019_Training_GroundTruth_folded.csv', index=False)
else:
    df.to_csv('data/ISIC_2019_Training_GroundTruth_folded.csv', index=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class MyCustomDataset(Dataset):
    def __init__(self, folder, fold, type, transform=None):
        self.type = type
        if self.type == 'train':
            self.df = pd.read_csv('data/ISIC_2019_Training_GroundTruth_folded.csv')
            self.df = self.df[self.df['fold'] != fold]
        else:
            self.df = pd.read_csv('data/ISIC_2019_Training_GroundTruth_folded.csv')
            self.df = self.df[self.df['fold'] == fold]
        #self.classes = 
        self.folder     = folder
        self.transform  = transform
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, index):
        path = os.path.join(self.folder, self.df.iloc[index]['image'] + '.jpg')
        x = Image.open(path)
        
        if self.transform:
            x = self.transform(x)
        
        y = torch.tensor(self.df.iloc[index]['label'], dtype=torch.long)
        
        return x, y

In [ ]:
tmfs = {'train' : transforms.Compose([
                        transforms.RandomRotation(45, resample=False, expand=False, center=None, fill=None),
                        transforms.RandomVerticalFlip(p=0.5),
                        transforms.Resize((224,224), interpolation=2),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
                   ]),
        'test' : transforms.Compose([
                        transforms.Resize((224,224), interpolation=2),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
                   ])
       }

In [ ]:
def train_model(model, dataloader, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train': model.train()
            else               : model.eval()
                
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def main():
    datadir = r'D:\skincancer\data\ISIC_2019_Training_Input\ISIC_2019_Training_Input'
    out_models = {}
    for fold in range(1, fold_n+1):
        print(f'Fold : {fold}')
        transformed_dataset = {e:MyCustomDataset(datadir, fold, e, transform=tmfs[e]) for e in ['train','test']}
        dataloader          = {e:DataLoader(transformed_dataset[e], batch_size=16, shuffle=True, num_workers=0) for e in ['train','test']}
        dataset_sizes       = {e: len(dataloader[e]) for e in ['train', 'test']}

        model     = models.resnet18(pretrained=True)
        num_ftrs  = model.fc.in_features
        model.fc  = nn.Linear(num_ftrs, 8)
        model     = model.to(device)
        criterion = nn.CrossEntropyLoss()

        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)    

        model = train_model(model, dataloader, criterion, optimizer, exp_lr_scheduler, num_epochs=1)
        out_models[f'Fold_{fold}'] = model
        break

In [ ]:
if __name__=='__main__':
    main()

In [ ]:
from torchsample.callbacks import EarlyStopping